In [65]:
from imutils import paths
import argparse
import time
import sys
import cv2
import os
import numpy as np
from PIL import Image

In [40]:
!ls ACACAURI | wc -l

     217


In [68]:
def noisy(noise_typ,image):
     print(noise_typ)
     if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
     elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
     elif noise_typ == "poisson":
        print("here")
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
     elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [73]:
dup = np.asarray(Image.open("needles/dup.jpg"))
res = noisy("poisson", dup).astype(np.uint8)
res_img = Image.fromarray(res)
res_img.save("needles/noisy_poisson.jpg")

here


In [80]:
def dhash(image, hashSize=8):
	# resize the input image, adding a single column (width) so we
	# can compute the horizontal gradient
	resized = cv2.resize(image, (hashSize + 1, hashSize))
	# compute the (relative) horizontal gradient between adjacent
	# column pixels
	diff = resized[:, 1:] > resized[:, :-1]
	# convert the difference image to a hash
	return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

In [83]:
print("[INFO] computing hashes for haystack...")
haystackPaths = list(paths.list_images("ACACAURI"))
needlePaths = list(paths.list_images("needles"))

[INFO] computing hashes for haystack...


In [84]:
# grab the base subdirectories for the needle paths, initialize the
# dictionary that will map the image hash to corresponding image,
# hashes, then start the timer
BASE_PATHS = set([p.split(os.path.sep)[-2] for p in needlePaths])
haystack = {}
start = time.time()

In [85]:
# loop over the haystack paths
for p in haystackPaths:
	# load the image from disk
	image = cv2.imread(p)
	# if the image is None then we could not load it from disk (so
	# skip it)
	if image is None:
		continue
	# convert the image to grayscale and compute the hash
	image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	imageHash = dhash(image)
	# update the haystack dictionary
	l = haystack.get(imageHash, [])
	l.append(p)
	haystack[imageHash] = l

In [86]:
# loop over the needle paths
count = 0
for p in needlePaths:
    # load the image from disk
    image = cv2.imread(p)
    # if the image is None then we could not load it from disk (so
    # skip it)
    if image is None:
        continue
    # convert the image to grayscale and compute the hash
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imageHash = dhash(image)
    # grab all image paths that match the hash
    matchedPaths = haystack.get(imageHash, [])
    # loop over all matched paths
    for matchedPath in matchedPaths:
        
        # extract the subdirectory from the image path
        b = p.split(os.path.sep)[-2]
        # if the subdirectory exists in the base path for the needle
        # images, remove it
        if b in BASE_PATHS:
            count += 1
print(count)

218


In [39]:
# display directories to check
print("[INFO] check the following directories...")
# loop over each subdirectory and display it
for b in BASE_PATHS:
	print("[INFO] {}".format(b))

[INFO] check the following directories...
[INFO] ACACAURI
